In [1]:
import os
import cv2
import glob
from tqdm import tqdm
import random

In [2]:
def upscale_image(image_path, target_width):
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if image.shape[0] != target_width or image.shape[1] != target_width:
        image = cv2.resize(image, (target_width, target_width), interpolation=cv2.INTER_LANCZOS4)
    return image

def upscale_images_in_folder(folder_path, target_width, path_pretrained, model_name):
    files_in_folder = glob.glob(os.path.join(folder_path, '*'))

    # Remove non-png files
    #for file_path in files_in_folder:
    #    if not file_path.endswith('.png'):
    #        os.remove(file_path)

    png_files = glob.glob(os.path.join(folder_path, '*.png'))
    for png_file in png_files:
        image = upscale_image(png_file, target_width)
        new_filename = "t" + str(os.path.basename(png_file).split('.')[0]).zfill(3) + '.tif'
        cv2.imwrite(os.path.join(folder_path, new_filename), image)
        os.remove(png_file)
    
    #tif_files = glob.glob(os.path.join(folder_path, '*.tif'))

    # Randomly pick 2 .tif files
    #random_selected_files = random.sample(tif_files, 2)

    print('cellpose current path',folder_path)
    # After processing the folder, run the Cellpose command
    os.system(f"python -m cellpose --dir '{folder_path}' --pretrained_model '{path_pretrained + model_name}' --save_png")
    #for selected_file in random_selected_files:
        #print('cellpose current path',selected_file)
        #os.system(f"python -m cellpose --img '{selected_file}' --pretrained_model '{path_pretrained + model_name}' --no_gui --save_png")

    # Clean up after Cellpose
    npy_files = glob.glob(os.path.join(folder_path, '*.npy'))
    for npy_file in npy_files:
        os.remove(npy_file)

    output_png_files = glob.glob(os.path.join(folder_path, '*_output.png'))
    for output_png in output_png_files:
        os.remove(output_png)

    mask_files = glob.glob(os.path.join(folder_path, '*_masks.png'))
    seg_folder_path = os.path.join(os.path.dirname(folder_path), os.path.basename(folder_path) + "_ST", "SEG")
    os.makedirs(seg_folder_path, exist_ok=True)
    for mask_file in mask_files:
        filename = os.path.basename(mask_file)  # Extract only the filename from the full path
        new_name = "man_seg" + filename.split('_')[0][1:] + ".tif"

        # Load the PNG image
        image = cv2.imread(mask_file, cv2.IMREAD_UNCHANGED)

        # Save the image as a TIFF file
        cv2.imwrite(os.path.join(seg_folder_path, new_name), image)

        # Remove the original PNG file
        os.remove(mask_file)
def process_directory(base_directory, target_width=1024, path_pretrained='', model_name=''):
    sub_folders = [name for name in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, name))]
    for sub_folder in tqdm(sorted(sub_folders)):
        if not sub_folder.endswith("_GT"):
            upscale_images_in_folder(os.path.join(base_directory, sub_folder), target_width, path_pretrained, model_name)

In [ ]:
# Example usage:
print(stop)
base_directory_path = '/media/mo/KINGSTON/Generated/HUH'  # Change this to your actual directory
path_pretrained = '/media/mo/KINGSTON/cellpose_train/HUH/models/'    # Change to the actual path
model_name = 'cellpose_residual_on_style_on_concatenation_off_HUH_2023_10_17_19_44_40.452432'                        # Change to the actual model name
target_width_value = 1024
process_directory(base_directory_path, target_width_value, path_pretrained, model_name)

In [ ]:
def process_masks(segmentation_mask_path, detection_gt_path, radius=10):
    # Load the images
    mask_img = cv2.imread(segmentation_mask_path, cv2.IMREAD_GRAYSCALE)
    gt_img = cv2.imread(detection_gt_path, cv2.IMREAD_GRAYSCALE)

    # Ensure the images are of the same shape
    if mask_img.shape != gt_img.shape:
        raise ValueError("Segmentation mask and ground truth images must have the same shape.")

    unique_masks = np.unique(mask_img)

    # For each unique value in segmentation mask, check if it overlaps with the detection ground truth
    for mask_val in unique_masks:
        if mask_val == 0:  # Ignore background
            continue
        overlap = np.sum((mask_img == mask_val) & (gt_img > 0))
        if overlap == 0:  # If no overlap, remove the entire mask
            mask_img[mask_img == mask_val] = 0

    # For each detection ground truth circle, if no overlapping mask, draw a new circle
    y, x = np.where(gt_img > 0)
    for i in range(len(x)):
        if mask_img[y[i], x[i]] == 0:
            new_label = mask_img.max() + 1  # Get a new unique label
            cv2.circle(mask_img, (x[i], y[i]), radius, new_label, -1)

    return mask_img

# Test the function
corrected_mask = process_masks('path_to_segmentation_mask.tif', 'path_to_detection_ground_truth.tif', radius=10)
cv2.imwrite('corrected_mask.tif', corrected_mask.astype(np.uint8))